In [ ]:
import pandas as pd
import numpy as np

# Project Title

create a dashboard where you can perform linear regression between 2 different variables
gas / electricity vs different economic sectors in GB

In [ ]:
#energy prices using CPI index over the years
# https://www.ons.gov.uk/economy/inflationandpriceindices/timeseries/d7du/mm23
# https://www.ons.gov.uk/economy/inflationandpriceindices/timeseries/d7dt/mm23
df_gas = pd.read_csv(filepath_or_buffer = "../data/raw/series-050923-gas.csv", header = 7, names = ['year', 'cpi_gas']) 
df_electricity = pd.read_csv(filepath_or_buffer = "../data/raw/series-160823-electricity.csv", header = 7, names = ['year', 'cpi_gas'] )

In [ ]:
df_electricity_months = df_electricity[177:].copy()
df_electricity_months['date_datetime'] = pd.to_datetime(df_electricity_months['year'], format='%Y %b')
df_electricity_months.to_csv("../data/processed/energy_price_electricity.csv")

In [ ]:
df_gas_months = df_gas[177:].copy()
df_gas_months['date_datetime'] = pd.to_datetime(df_gas_months['year'], format='%Y %b')
#df_gas_quarters = df_gas[35:177].copy()
#df_gas_years = df_gas[:35].copy()

In [ ]:
df_gas_months.head()

In [ ]:
df_CPI = pd.read_csv(filepath_or_buffer = "../data/raw/mm23.csv"
                     )

In [ ]:
#getting the right months
df_months = df_CPI[1017:1444].copy()
df_months['Title'] = pd.to_datetime(df_months['Title'], format='%Y %b')

In [ ]:
#df_electricity_cpi = df_months.loc[:, ['Title', 'CPI INDEX 04.5.1 : ELECTRICITY 2015=100']]
#df_gas_cpi = df_months.loc[:, ['Title', 'CPI INDEX 04.5.2 : GAS 2015=100']]
#df_clothing_footwear_cpi = df_months.loc[:, ['Title', 'CPI INDEX 03 : CLOTHING AND FOOTWEAR 2015=100']]
#df_alcoholic_tobacco = df_months.loc[:, ['Title', "CPI INDEX 02:ALCOHOLIC BEVERAGES,TOBACCO & NARCOTICS 2015=100"]]
#df_education = df_months.loc[:, ['Title', "CPI INDEX 10 : EDUCATION 2015=100"]]
#df_misc = df_months.loc[:, ['Title', "CPI INDEX 12 : MISCELLANEOUS GOODS AND SERVICES 2015=100"]]
#df_communication = df_months.loc[:, ['Title', "CPI INDEX 08 : COMMUNICATION 2015=100"]]
#df_furniture = df_months.loc[:, ['Title', "CPI INDEX 05.1.1 : Furniture and furnishings 2015=100"]]
#df_health = df_months.loc[:, ['Title', "CPI INDEX 06 : HEALTH 2015=100"]]
#df_recreation = df_months.loc[:, ['Title', "CPI INDEX 09 : RECREATION & CULTURE 2015=100"]]
#df_food = df_months.loc[:, ['Title', "CPI INDEX 01 : FOOD AND NON-ALCOHOLIC BEVERAGES 2015=100" ]]
#df_restaurants = df_months.loc[:, ['Title', "CPI INDEX 11 : HOTELS, CAFES AND RESTAURANTS 2015=100"]]
#df_transport = df_months.loc[:, ['Title', "CPI INDEX 07 : TRANSPORT 2015=100" ]]
#df_housing_rent = df_months.loc[:, ['Title', "CPI INDEX 04.1 : ACTUAL RENTS FOR HOUSING 2015=100"]]
#df_housing_rent_private = df_CPI.loc[1017:1444, ['Title', "CPI INDEX: PRIVATE RENTALS FOR HOUSING 2015=100"]]
#df_housing_rent_non_private=df_CPI.loc[1017:1444, ['Title', "CPI INDEX: NON - PRIVATE RENTALS FOR HOUSING 2015=100"]]
#df_housing_rent_social=df_CPI.loc[1017:1444, ['Title', "CPI INDEX: REGISTERED SOCIAL LANDLORD RENTALS FOR HOUSING 2015=100"]]
#df_housing_rent_la =df_CPI.loc[1017:1444, ['Title',  "CPI INDEX: LOCAL AUTHORITY RENTALS FOR HOUSING 2015=100"]]                          

In [ ]:
CPI_division_level = [
    'CPI INDEX 04.5.1 : ELECTRICITY 2015=100',
    'CPI INDEX 04.5.2 : GAS 2015=100',
    'CPI INDEX 03 : CLOTHING AND FOOTWEAR 2015=100',
    "CPI INDEX 02:ALCOHOLIC BEVERAGES,TOBACCO & NARCOTICS 2015=100",
    "CPI INDEX 10 : EDUCATION 2015=100",
    "CPI INDEX 12 : MISCELLANEOUS GOODS AND SERVICES 2015=100",
    "CPI INDEX 08 : COMMUNICATION 2015=100",
    "CPI INDEX 05.1 : Furniture, furnishings and carpets 2015=100",
    "CPI INDEX 06 : HEALTH 2015=100",
    "CPI INDEX 09 : RECREATION & CULTURE 2015=100",
    "CPI INDEX 01 : FOOD AND NON-ALCOHOLIC BEVERAGES 2015=100",
    "CPI INDEX 11 : HOTELS, CAFES AND RESTAURANTS 2015=100",
    "CPI INDEX 07 : TRANSPORT 2015=100",
    "CPI INDEX 04.1 : ACTUAL RENTS FOR HOUSING 2015=100"
]

# Create a list of DataFrames to merge
dfs_to_merge = [df_months[['Title', x]] for x in CPI_division_level]

# Merge the DataFrames on the "Title" column
merged_df = dfs_to_merge[0]
for df in dfs_to_merge[1:]:
    merged_df = pd.merge(merged_df, df, on="Title", how='inner')



columns = ['Date', 'Electricity', 'Gas', 'Clothing and Footwear', 'Alcoholic Beverages, tobacco & Narcotics', 'Education',
                       'Misc Goods and Services', 'Communication', 'Furniture and Furnishings', 'Health', 'Recreation & Culture',
                       'Food and Non-Alcoholic Beverages', 'Hotels, Cafes and Restaurants', 'Transport', 'Actual Rents for Housing']
merged_df.columns = columns

In [ ]:
merged_df_long = merged_df.copy()
merged_df_long = merged_df_long.set_index('Date').melt(ignore_index=False).reset_index()
merged_df_long

In [ ]:
merged_df.to_csv("../data/processed/merged_df.csv")
merged_df_long.to_csv("../data/processed/merged_df_long.csv")